In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers,models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Paths
train_path = '/content/drive/MyDrive/Datasets/train'
test_path = '/content/drive/MyDrive/Datasets/test'
valid_path = '/content/drive/MyDrive/Datasets/valid'

In [ ]:
# Image size and batch
IMG_SIZE = (224, 224) #Height, width in pixels
BATCH_SIZE = 32 #Number of images to be processed...

In [ ]:
# Data generators for train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

#Data generators for valid
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Data generators for test
test_datagen = ImageDataGenerator(rescale=1./255)

Train data needs variety (augment), Test data needs clarity (just rescale)
<br>
These tricks help your model see different versions of the same image — this is called Data Augmentation. It makes the model stronger and better at generalizing.

For test data, we don’t use flips or rotations — we just rescale. Test data should be clean.

In [ ]:
train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_gen = valid_datagen.flow_from_directory(
    valid_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 1695 images belonging to 4 classes.
Found 246 images belonging to 4 classes.
Found 512 images belonging to 4 classes.


MobileNetV2: A famous model trained on millions of images (ImageNet) — it already knows useful patterns like edges, textures, etc.

In [ ]:
# Transfer Learning Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

#model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), #Reduces data from base model into a flat feature vector.
    layers.Dense(128, activation='relu'), #Fully connected layer — learns complex patterns.
    layers.Dropout(0.3), #Turns off 30% neurons randomly during training to avoid overfitting.
    layers.Dense(train_gen.num_classes, activation='softmax')#Final layer — gives class probabilities.
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Adam optimizer (a fast learner)
The loss function — categorical_crossentropy for multi-class classification.

In [ ]:
# Train
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=10
)

Epoch 1/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8919 - loss: 0.2995

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


53/53 ━━━━━━━━━━━━━━━━━━━━ 229s 4s/step - accuracy: 0.8918 - loss: 0.2997 - val_accuracy: 0.8398 - val_loss: 0.4570
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.8856 - loss: 0.3130 - val_accuracy: 0.8359 - val_loss: 0.4668
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.8856 - loss: 0.3058 - val_accuracy: 0.8652 - val_loss: 0.3866
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.9104 - loss: 0.2236 - val_accuracy: 0.8828 - val_loss: 0.3398
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.9114 - loss: 0.2227 - val_accuracy: 0.8633 - val_loss: 0.4086
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.9280 - loss: 0.1968 - val_accuracy: 0.8477 - val_loss: 0.4310
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.9309 - loss: 0.1991 - val_accuracy: 0.8438 - val_loss: 0.4667
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - accuracy: 0.9436 - loss: 0.1598 - val_accuracy: 0.8613 - val_loss: 0.

In [ ]:
# Save model
model.save('best_model.h5')

In [ ]:
# Save class names
with open('class_names.txt', 'w') as f:
    f.write('\n'.join(list(train_gen.class_indices.keys())))  #Saves the list of class names (like ['glioma', 'meningioma', 'pituitary']) to a text file.
    #Later when the model gives output like [0, 0, 1], you can convert it back to label name using this file.